Lecture 5 Notebook
------------------

_Note: This notebook mostly covers material from sections 2 and 3 of Lecture 5_

 * **Goals**: Learn keys, superkeys, and the FD closure algorithm.

Given a set of attributes  $\{A_1, \dots, A_n\}$ and a set of FDs $\Gamma$

The closure, denoted $\{A_1, \dots, A_n\}^+$, is defined to be the largest set of attributes B s.t. $$A_1,\dots,A_n \rightarrow B \text{ using } \Gamma.$$

_First, we'll set up some utility functions quickly:_

In [3]:
A  = set(["name", "category"]) # These are the attribute set.
fds = [ (set(["name"]),"color"), # name -> color
        (set(["category"]), "department"), # category -> department
        (set(["color", "category"]), "price") ] # color, category -> price
def to_set(x):
    if type(x) == set:
        return x
    elif type(x) in [list, set]:
        return set(x)
    elif type(x) in [str, int]:
        return set([x])
    else:
        raise Exception("Unrecognized type.")
def fd_to_str((lhs,rhs)): return ",".join(to_set(lhs)) + " -> " + ",".join(to_set(rhs))
def fds_to_str(fds): return "\n\t".join(map(fd_to_str, fds))
def set_to_str(x): return "{" + ",".join(x) + "}"
def print_setup(A, fds):
    print("Attributes = " + set_to_str(A))
    print("FDs = \t" + fds_to_str(fds))

In [4]:
print_setup(A, fds)

Attributes = {category,name}
FDs = 	name -> color
	category -> department
	color,category -> price


Now we define a simple function to determing if a FD $f = (lhs, rhs)$ **_applies to_** a set of attributes $x$:

In [5]:
def fd_applies_to((lhs,rhs), x): return to_set(lhs).issubset(x)

In [6]:
print_setup(A, fds); print "\n" + fd_to_str(fds[0]) + " applies to attributes set? " + str(fd_applies_to(fds[0],A)); 

Attributes = {category,name}
FDs = 	name -> color
	category -> department
	color,category -> price

name -> color applies to attributes set? True


In [8]:
print fd_to_str(fds[2]) + " applies to attributes set? " + str(fd_applies_to(fds[2],A));

color,category -> price applies to attributes set? False


**Note** also that although our functions work for general FDs of form $\{a_1,...,a_n\}\rightarrow\{b_1,...b_m\}$, for convenience, **we only actually need to support FDs of the form $\{a_1,...,a_n\}\rightarrow b$**.  Why?  See lecture!

### An algorithm for Computing Closures:

In [9]:
def compute_closure(x, fds, verbose=False):
    bChanged = True        # We will repeat until there are no changes.
    x_ret    = x.copy()    # Make a copy of the input to hold x^{+}
    while bChanged:
        bChanged = False   # Must change on each iteration
        for fd in fds:     # loop through all the FDs.
            (lhs, rhs) = map(to_set, fd) # recall: lhs -> rhs
            if fd_applies_to(fd, x_ret) and not rhs.issubset(x_ret):
                x_ret = x_ret.union(rhs)
                if verbose:
                    print("Using FD " + fd_to_str(fd))
                    print("\t Updated x to " + set_to_str(x_ret))
                bChanged = True
    return x_ret

In [10]:
print_setup(A, fds)
A_plus = compute_closure(A, fds, verbose=True)
print '\n' + set_to_str(A) + "+ = " + set_to_str(A_plus)

Attributes = {category,name}
FDs = 	name -> color
	category -> department
	color,category -> price
Using FD name -> color
	 Updated x to {category,color,name}
Using FD category -> department
	 Updated x to {category,color,department,name}
Using FD color,category -> price
	 Updated x to {category,color,price,department,name}

{category,name}+ = {category,color,price,department,name}


*Another example:*

In [11]:
fds = [ (set(["A", "B"]), "C"), (set(["A", "D"]), "E"), 
        (set(["B"])     , "D"), (set(["A","F"]) , "B")] 
print "FDs:\t" + fds_to_str(fds)

FDs:	A,B -> C
	A,D -> E
	B -> D
	A,F -> B


In [12]:
# Compute {A,B}+
A0 = set(["A", "B"])
A0_plus = compute_closure(A0, fds, verbose=True)
print '\n' + set_to_str(A0) + "+ = " + set_to_str(A0_plus)

Using FD A,B -> C
	 Updated x to {A,C,B}
Using FD B -> D
	 Updated x to {A,C,B,D}
Using FD A,D -> E
	 Updated x to {A,C,B,E,D}

{A,B}+ = {A,C,B,E,D}


In [13]:
# Compute {A,F}+
A1 = set(["A", "F"])
A1_plus = compute_closure(A1, fds, verbose=True)
print '\n' + set_to_str(A1) + "+ = " + set_to_str(A1_plus)

Using FD A,F -> B
	 Updated x to {A,B,F}
Using FD A,B -> C
	 Updated x to {A,C,B,F}
Using FD B -> D
	 Updated x to {A,C,B,D,F}
Using FD A,D -> E
	 Updated x to {A,C,B,E,D,F}

{A,F}+ = {A,C,B,E,D,F}


### Using FD closures to check FDs:

With the FD closure, we can check FDs easily!

(_Again, note that we only need to check set-to-single-attribute FDs, though our functions support set-to-set ones!_)

For attribute set $X$, attribute $a$:
* To check if $X \rightarrow a$
  * Compute $X^{+}$
  * Check if $a \in X^{+}$
      * If yes $\implies X \rightarrow a$!
      
Why?  A: Transitivity + trivial (+ combine) rules!

In [14]:
def is_fd_implied(fds, lhs, rhs, verbose=False):
   xp = compute_closure(lhs,fds,verbose=verbose)
   if verbose: print(set_to_str(lhs) +"+ = "+ set_to_str(xp))
   return to_set(rhs).issubset(xp)

In [15]:
# Check if AB -> E
is_fd_implied(fds, set(["A","B"]), "E", verbose=True)

Using FD A,B -> C
	 Updated x to {A,C,B}
Using FD B -> D
	 Updated x to {A,C,B,D}
Using FD A,D -> E
	 Updated x to {A,C,B,E,D}
{A,B}+ = {A,C,B,E,D}


True

In [16]:
# Check if AB -> F
is_fd_implied(fds, set(["A","B"]), "F", verbose=True)

Using FD A,B -> C
	 Updated x to {A,C,B}
Using FD B -> D
	 Updated x to {A,C,B,D}
Using FD A,D -> E
	 Updated x to {A,C,B,E,D}
{A,B}+ = {A,C,B,E,D}


False

### Superkeys and Keys

* A _superkey_ for a relation $R$ having attributes $B = \{B_1,\dots,B_m\}$ is a set of attributes $A\subseteq B$ s.t. $A\rightarrow B$

* A _key_ is a minimal (setwise) _superkey_

In [17]:
# check if A is a superkey for attributes B according to FDs F
def is_superkey_slow(A,B,F):
    return all([is_fd_implied(F,A,b) for b in B])

How can we check if a set of attributes $A$ is a superkey for $R$ with attributes $B$, given FDs $F$?

### Simple algorithm using the FD / closure algorithms that we made:

In [18]:
def is_superkey(A, B, F): return is_fd_implied(F, A, B)

In [19]:
B=set(["A","B","C","D","E","F"])
A=set(["A","B","F"])
is_superkey(A, B, fds)

True

In [20]:
B=set(["A","B","C","D","E","F"])
is_superkey_slow(set(["A","B","F"]), B, fds)

True

In [21]:
# This is equivalent
def is_superkey(A,B,fds, verbose=False): 
    return B.issubset(compute_closure(A,fds,verbose=verbose))

In [22]:
is_superkey(set(["A","B","F"]), B, fds,verbose=True)

Using FD A,B -> C
	 Updated x to {A,C,B,F}
Using FD B -> D
	 Updated x to {A,C,B,D,F}
Using FD A,D -> E
	 Updated x to {A,C,B,E,D,F}


True

Then, to test if $A$ is a _key_, need to check that:
1. $A$ is a super key
2. No proper subset of $A$ is a superkey (or a key)

Why is it enough to check just sets of the form $A \setminus \{a\}$ for $a \in A$?

In [25]:
import itertools
def is_key(A,B,fds,verbose=False):
    subsets = set(itertools.combinations(A, len(A)-1))
    return is_superkey(A,B,fds) and \
        all([not is_superkey(set(SA),B,fds) for SA in subsets])

In [26]:
print_setup(B, fds)
C = set(["A","F","B"])
C_1 = set(["A", "F"])
print set_to_str(C) + " is superkey?  " + str(is_superkey(C, B, fds))
print set_to_str(C) + " is key?  " + str(is_key(C, B, fds))
print set_to_str(C_1) + " is key?  " + str(is_key(C_1, B, fds))

Attributes = {A,C,B,E,D,F}
FDs = 	A,B -> C
	A,D -> E
	B -> D
	A,F -> B
{A,B,F} is superkey?  True
{A,B,F} is key?  False
{A,F} is key?  True


Can you create a relation with two keys?

Can you create a relation and FDs with two keys?
* Audience participation required...

In this notebook, we learned about
* functional dependencies and their axioms
* the definitions of _super_ _key_ and _key_
* how to check these definisions using the FD closure.